In [1]:
import pandas as pd
import numpy as np

#导入feauretool
import featuretools as ft

In [2]:
#加载数据
clients = pd.read_csv('data/clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('data/loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('data/payments.csv', parse_dates = ['payment_date'])

In [3]:
clients.head()

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [5]:
loans.head()

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
0,46109,home,13672,0,10243,2002-04-16,2003-12-20,2.15
1,46109,credit,9794,0,10984,2003-10-21,2005-07-17,1.25
2,46109,home,12734,1,10990,2006-02-01,2007-07-05,0.68
3,46109,cash,12518,1,10596,2010-12-08,2013-05-05,1.24
4,46109,credit,14049,1,11415,2010-07-07,2012-05-21,3.13


In [6]:
payments.head()

,loan_id,payment_amount,payment_date,missed
0,10243,2369,2002-05-31,1
1,10243,2439,2002-06-18,1
2,10243,2662,2002-06-29,0
3,10243,2268,2002-07-20,0
4,10243,2027,2002-07-31,1


In [7]:
#创建实体
es = ft.EntitySet(id = 'clients')

#添加clients实体
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

#添加loads实体
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = {'repaid': ft.variable_types.Categorical},
                              index = 'loan_id', 
                              time_index = 'loan_start')


#添加pyments实体
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')
#打印实体集
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [8]:
#添加实体关系
# 通过client_id 关联clients和loans实体
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])
es = es.add_relationship(r_client_previous)

# 通过loan_id 关联payments和loans实体
r_payments = ft.Relationship(es['loans']['loan_id'],
                             es['payments']['loan_id'])
es = es.add_relationship(r_payments)

#打印实体集
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

In [16]:
#聚合特征,并生成新特征
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients')
features.head()

,income,credit_score,SUM(loans.loan_amount),SUM(loans.rate),STD(loans.loan_amount),STD(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),SKEW(loans.loan_amount),SKEW(loans.rate),...,NUM_UNIQUE(loans.WEEKDAY(loan_end)),MODE(loans.MODE(payments.missed)),MODE(loans.DAY(loan_start)),MODE(loans.DAY(loan_end)),MODE(loans.YEAR(loan_start)),MODE(loans.YEAR(loan_end)),MODE(loans.MONTH(loan_start)),MODE(loans.MONTH(loan_end)),MODE(loans.WEEKDAY(loan_start)),MODE(loans.WEEKDAY(loan_end))
client_id,,,,,,,,,,,,,,,,,,,,,
25707,211422,621,159279,69.54,4044.418728,2.421285,13913,9.44,-0.172074,0.679118,...,6,0,27,1,2010,2007,1,8,3,0
26326,227920,633,116321,40.28,4254.149422,1.991819,13464,6.73,0.135246,1.067853,...,5,0,6,6,2003,2005,4,7,5,2
26695,174532,680,140845,44.39,4078.228493,1.517660,14865,6.51,0.154467,0.820060,...,6,0,3,14,2003,2005,9,4,1,1
26945,214516,806,106889,42.83,4389.555657,1.564795,14593,5.65,0.156534,-0.001998,...,6,0,16,1,2002,2004,12,5,0,1
29841,38354,523,176634,62.01,4090.630609,2.063092,14837,6.76,-0.212397,0.050600,...,7,1,1,15,2005,2007,3,2,5,1


In [17]:
#聚合特征，通过指定聚合agg_primitives和转换trans_primitives生成新特征
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['years', 'month', 'subtract', 'divide'])
features.head()

,income,credit_score,MEAN(loans.loan_amount),MEAN(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),LAST(loans.loan_type),LAST(loans.loan_amount),LAST(loans.rate),LAST(loans.repaid),...,MEAN(loans.loan_amount) / credit_score,LAST(payments.payment_amount) / MEAN(loans.loan_amount),MEAN(loans.loan_amount) / MEAN(loans.rate),LAST(loans.rate) / LAST(payments.payment_amount),LAST(loans.loan_amount) / income - credit_score,LAST(payments.payment_amount) / MEAN(loans.rate),MAX(loans.rate) / LAST(loans.rate),MAX(payments.payment_amount) / income,credit_score - income / credit_score,credit_score - income / MAX(loans.rate)
client_id,,,,,,,,,,,,,,,,,,,,,
25707,211422,621,7963.950000,3.477000,13913,9.44,home,2203,7.40,0,...,12.824396,0.030261,2290.465919,0.030705,0.010451,69.312626,1.275676,0.012790,-339.454106,-22330.614407
26326,227920,633,7270.062500,2.517500,13464,6.73,credit,5275,1.45,0,...,11.485091,0.128197,2887.810328,0.001556,0.023209,370.208540,4.641379,0.011662,-359.063191,-33772.213967
26695,174532,680,7824.722222,2.466111,14865,6.51,other,13918,0.90,1,...,11.506944,0.266335,3172.899302,0.000432,0.080057,845.055193,7.233333,0.016799,-255.664706,-26705.376344
26945,214516,806,7125.933333,2.855333,14593,5.65,cash,9249,2.86,1,...,8.841108,0.224111,2495.657250,0.001791,0.043278,559.304226,1.975524,0.012903,-265.148883,-37824.778761
29841,38354,523,9813.000000,3.445000,14837,6.76,home,7223,5.09,1,...,18.762906,0.081525,2848.476052,0.006363,0.190928,232.220610,1.328094,0.075559,-72.334608,-5596.301775
